In [1]:
!rm cz_2010.sqlite

rm: cz_2010.sqlite: No such file or directory


In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import *
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [3]:
df0 = pd.read_csv('../data_in/LOS-ANGELES-DOWNTOWN_722874_CZ2010.CSV', low_memory=False)
df1 = pd.read_csv('../data_in/LONG-BEACH_722970_CZ2010.CSV', low_memory=False)
df2 = pd.read_csv('../data_in/BURBANK-GLENDALE_722880_CZ2010.CSV', low_memory=False)

In [4]:
df0.shape

(8760, 69)

In [5]:
df0['TYPE'] = 'CZ_2010'
df0['STATION'] = 'LOS-ANGELES-DOWNTOWN-USC'
df0['STATION_NBR'] = '722874'

df1['TYPE'] = 'CZ_2010'
df1['STATION'] = 'LONG-BEACH'
df1['STATION_NBR'] = '722970'

df2['TYPE'] = 'CZ_2010'
df2['STATION'] = 'BURBANK-GLENDALE'
df2['STATION_NBR'] = '722880'

In [6]:
df = pd.concat([df0, df1, df2])

In [7]:
df.shape

(26280, 72)

In [8]:
df.head()

,Date (MM/DD/YYYY),Time (HH:MM),ETR (W/m^2),ETRN (W/m^2),GHI (W/m^2),GHI source,GHI uncert (%),DNI (W/m^2),DNI source,DNI uncert (%),...,Alb source,Alb uncert (code),Lprecip depth (mm),Lprecip quantity (hr),Lprecip source,Lprecip uncert (code),PresWth (METAR code),TYPE,STATION,STATION_NBR
0,2004-01-01,1:00,0,0,0,?,0,0,?,0,...,NaN,NaN,0,1,B,8,0,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
1,2004-01-01,2:00,0,0,0,?,0,0,?,0,...,NaN,NaN,0,1,?,0,45,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
2,2004-01-01,3:00,0,0,0,?,0,0,?,0,...,NaN,NaN,0,1,?,0,5,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
3,2004-01-01,4:00,0,0,0,?,0,0,?,0,...,NaN,NaN,0,1,B,8,5,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
4,2004-01-01,5:00,0,0,0,?,0,0,?,0,...,NaN,NaN,0,1,B,8,45,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874


In [9]:
df['date_time'] = df['Date (MM/DD/YYYY)'] + ' ' + df['Time (HH:MM)']

In [10]:
df.columns = df.columns.str.upper()

In [11]:
df.columns.tolist()

['DATE (MM/DD/YYYY)',
 'TIME (HH:MM)',
 'ETR (W/M^2)',
 'ETRN (W/M^2)',
 'GHI (W/M^2)',
 'GHI SOURCE',
 'GHI UNCERT (%)',
 'DNI (W/M^2)',
 'DNI SOURCE',
 'DNI UNCERT (%)',
 'DHI (W/M^2)',
 'DHI SOURCE',
 'DHI UNCERT (%)',
 'GH ILLUM (LX)',
 'GH ILLUM SOURCE',
 'GLOBAL ILLUM UNCERT (%)',
 'DN ILLUM (LX)',
 'DN ILLUM SOURCE',
 'DN ILLUM UNCERT (%)',
 'DH ILLUM (LX)',
 'DH ILLUM SOURCE',
 'DH ILLUM UNCERT (%)',
 'ZENITH LUM (CD/M^2)',
 'ZENITH LUM SOURCE',
 'ZENITH LUM UNCERT (%)',
 'TOTCLD (TENTHS)',
 'TOTCLD SOURCE',
 'TOTCLD UNCERT (CODE)',
 'OPQCLD (TENTHS)',
 'OPQCLD SOURCE',
 'OPQCLD UNCERT (CODE)',
 'DRY-BULB (C)',
 'DRY-BULB SOURCE',
 'DRY-BULB UNCERT (CODE)',
 'DEW-POINT (C)',
 'DEW-POINT SOURCE',
 'DEW-POINT UNCERT (CODE)',
 'RHUM (%)',
 'RHUM SOURCE',
 'RHUM UNCERT (CODE)',
 'PRESSURE (MBAR)',
 'PRESSURE SOURCE',
 'PRESSURE UNCERT (CODE)',
 'WDIR (DEGREES)',
 'WDIR SOURCE',
 'WDIR UNCERT (CODE)',
 'WSPD (M/S)',
 'WSPD SOURCE',
 'WSPD UNCERT (CODE)',
 'HVIS (M)',
 'HVIS SOURCE',

In [12]:
df_subset = df[['DATE (MM/DD/YYYY)', 'TIME (HH:MM)', 'TYPE', 'STATION_NBR', 'STATION', 'DRY-BULB (C)']]

In [13]:
df_subset.columns = df_subset.columns.str.replace('(', '')
df_subset.columns = df_subset.columns.str.replace(')', '')
df_subset.columns = df_subset.columns.str.replace(' ', '_')
df_subset.columns = df_subset.columns.str.replace('/', '_')
df_subset.columns = df_subset.columns.str.replace('-', '_')
df_subset.columns = df_subset.columns.str.replace(':', '_')

In [14]:
df_subset.head()

,DATE_MM_DD_YYYY,TIME_HH_MM,TYPE,STATION_NBR,STATION,DRY_BULB_C
0,2004-01-01,1:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,9.0
1,2004-01-01,2:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.8
2,2004-01-01,3:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.8
3,2004-01-01,4:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.2
4,2004-01-01,5:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,6.1


In [15]:
df_subset['TIME_DT_H'] = df_subset['TIME_HH_MM'].str.split(':').str[0]
df_subset['TIME_DT_HH'] = df_subset['TIME_DT_H'].apply(lambda x: x.zfill(2))
df_subset['TEMP_F'] = 'TEMP_F_'
df_subset['TIME_DT_M'] = '00'
df_subset['DRY_BULB_F'] = (df_subset['DRY_BULB_C'] * (9/5)) + 32
df_subset['TIMESTAMP'] = df_subset[['TEMP_F', 'TIME_DT_HH', 'TIME_DT_M']].apply(lambda x: ''.join(x), axis=1)

/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [16]:
df_subset.head()

,DATE_MM_DD_YYYY,TIME_HH_MM,TYPE,STATION_NBR,STATION,DRY_BULB_C,TIME_DT_H,TIME_DT_HH,TEMP_F,TIME_DT_M,DRY_BULB_F,TIMESTAMP
0,2004-01-01,1:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,9.0,1,01,TEMP_F_,00,48.20,TEMP_F_0100
1,2004-01-01,2:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.8,2,02,TEMP_F_,00,46.04,TEMP_F_0200
2,2004-01-01,3:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.8,3,03,TEMP_F_,00,46.04,TEMP_F_0300
3,2004-01-01,4:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,7.2,4,04,TEMP_F_,00,44.96,TEMP_F_0400
4,2004-01-01,5:00,CZ_2010,722874,LOS-ANGELES-DOWNTOWN-USC,6.1,5,05,TEMP_F_,00,42.98,TEMP_F_0500


In [17]:
FILTER = ['DATE_MM_DD_YYYY',
          'TIMESTAMP',
          'DRY_BULB_F',
          'TYPE',
          'STATION', 
          'STATION_NBR']

df = df_subset[FILTER]

In [18]:
df.head()

,DATE_MM_DD_YYYY,TIMESTAMP,DRY_BULB_F,TYPE,STATION,STATION_NBR
0,2004-01-01,TEMP_F_0100,48.20,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
1,2004-01-01,TEMP_F_0200,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
2,2004-01-01,TEMP_F_0300,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
3,2004-01-01,TEMP_F_0400,44.96,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
4,2004-01-01,TEMP_F_0500,42.98,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874


In [19]:
df_pivot = df.pivot_table(index=['DATE_MM_DD_YYYY', 
                                 'TYPE', 
                                 'STATION', 
                                 'STATION_NBR'], 
                          columns='TIMESTAMP',
                          values='DRY_BULB_F', 
                          aggfunc='sum')

In [20]:
#https://stackoverflow.com/questions/38951345/how-to-get-rid-of-multilevel-index-after-using-pivot-table-pandas
df_pivot = df_pivot.rename_axis(None, axis=1).reset_index() 

In [21]:
df_pivot.rename(columns={'DATE_MM_DD_YYYY': 'READ_DT'}, inplace=True)
df_pivot['READ_DT_MON'] = df_pivot['READ_DT'].str.split('-').str[1]
df_pivot['READ_DT_DAY'] = df_pivot['READ_DT'].str.split('-').str[2]
df_pivot['READ_DT_YEAR'] = df_pivot['READ_DT'].str.split('-').str[0]

In [22]:
df = df_pivot[[
         'READ_DT',
         'READ_DT_MON',
         'READ_DT_DAY',
         'READ_DT_YEAR',
         'TYPE',
         'STATION',
         'STATION_NBR',
         'TEMP_F_0000',
         'TEMP_F_0100',
         'TEMP_F_0200',
         'TEMP_F_0300',
         'TEMP_F_0400',
         'TEMP_F_0500',
         'TEMP_F_0600',
         'TEMP_F_0700',
         'TEMP_F_0800',
         'TEMP_F_0900',
         'TEMP_F_1000',
         'TEMP_F_1100',
         'TEMP_F_1200',
         'TEMP_F_1300',
         'TEMP_F_1400',
         'TEMP_F_1500',
         'TEMP_F_1600',
         'TEMP_F_1700',
         'TEMP_F_1800',
         'TEMP_F_1900',
         'TEMP_F_2000',
         'TEMP_F_2100',
         'TEMP_F_2200',
         'TEMP_F_2300']]

In [23]:
df.sort_values(by=['STATION_NBR', 'READ_DT_MON', 'READ_DT_DAY'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [24]:
df['READ_DT'] = pd.to_datetime(df['READ_DT'])

In [25]:
df.head()

,READ_DT,READ_DT_MON,READ_DT_DAY,READ_DT_YEAR,TYPE,STATION,STATION_NBR,TEMP_F_0000,TEMP_F_0100,TEMP_F_0200,...,TEMP_F_1400,TEMP_F_1500,TEMP_F_1600,TEMP_F_1700,TEMP_F_1800,TEMP_F_1900,TEMP_F_2000,TEMP_F_2100,TEMP_F_2200,TEMP_F_2300
0,2004-01-01,01,01,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,51.80,48.20,46.04,...,60.08,62.06,62.06,60.98,58.82,56.84,55.04,53.60,53.24,51.80
1,2004-01-02,01,02,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,53.60,51.80,51.98,...,51.08,51.98,51.98,51.98,51.80,53.60,53.60,53.60,52.16,53.60
2,2004-01-03,01,03,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,46.04,53.24,50.00,...,55.94,57.02,57.02,55.04,54.32,53.24,51.44,50.00,49.64,47.84
3,2004-01-04,01,04,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,44.60,44.24,42.08,...,59.00,60.98,60.08,57.02,55.04,53.24,51.08,47.84,46.40,46.04
4,2004-01-05,01,05,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,53.24,44.60,44.96,...,62.96,64.04,64.94,64.94,64.58,63.14,58.28,55.04,53.60,53.60


In [26]:
df.STATION.value_counts()

BURBANK-GLENDALE            365
LONG-BEACH                  365
LOS-ANGELES-DOWNTOWN-USC    365
Name: STATION, dtype: int64

In [27]:
#Create CSV for testing.
#df_722970 = df[df['STATION_NBR'] == '722970']
#df_722970.to_csv('station_722970.csv', index=False)

### Create SQL Database

In [28]:
engine = create_engine('sqlite:///cz_2010.sqlite')

In [29]:
conn = engine.connect()
Base = declarative_base()

class Data(Base):
    __tablename__ = 'cz_2010_usc'
    
    id = Column(Integer, primary_key=True)
    READ_DT = Column(DateTime)
    READ_DT_MON = Column(Integer)
    READ_DT_DAY = Column(Integer)
    READ_DT_YEAR = Column(Integer)
    TYPE = Column(String)
    STATION = Column(String)
    STATION_NBR = Column(Integer)
    TEMP_F_0000 = Column(Float)
    TEMP_F_0100 = Column(Float)
    TEMP_F_0200 = Column(Float)
    TEMP_F_0300 = Column(Float)
    TEMP_F_0400 = Column(Float)
    TEMP_F_0500 = Column(Float)
    TEMP_F_0600 = Column(Float)
    TEMP_F_0700 = Column(Float)
    TEMP_F_0800 = Column(Float)
    TEMP_F_0900 = Column(Float)
    TEMP_F_1000 = Column(Float)
    TEMP_F_1100 = Column(Float)
    TEMP_F_1200 = Column(Float)
    TEMP_F_1300 = Column(Float)
    TEMP_F_1400 = Column(Float)
    TEMP_F_1500 = Column(Float)
    TEMP_F_1600 = Column(Float)
    TEMP_F_1700 = Column(Float)
    TEMP_F_1800 = Column(Float)
    TEMP_F_1900 = Column(Float)
    TEMP_F_2000 = Column(Float)
    TEMP_F_2100 = Column(Float)
    TEMP_F_2200 = Column(Float)
    TEMP_F_2300 = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [30]:
Base.metadata.create_all(engine)

In [31]:
data_dict = df.to_dict(orient='records')

In [32]:
metadata = MetaData(bind = engine)
metadata.reflect()

In [33]:
table = sqlalchemy.Table('cz_2010_usc', metadata, autoload=True)

In [34]:
conn.execute(table.delete())

In [35]:
conn.execute(table.insert(), data_dict)

### Test Connection with an actual SQL Query

In [36]:
conn.execute("SELECT SUM(TEMP_F_0000) \
              FROM cz_2010_usc \
              WHERE READ_DT_MON=04").fetchall()

[(4992.66,)]

In [37]:
conn.execute("SELECT * \
              FROM cz_2010_usc \
              WHERE READ_DT_MON=04").fetchall()[0:5]

[(91, '2001-04-01 00:00:00.000000', 4, 1, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 57.019999999999996, 62.42, 60.44, 59.18, 58.1, 57.019999999999996, 57.019999999999996, 57.019999999999996, 57.019999999999996, 55.94, 57.019999999999996, 59.0, 60.980000000000004, 62.06, 60.980000000000004, 64.94, 64.03999999999999, 60.980000000000004, 59.0, 59.0, 57.92, 57.92, 57.92, 57.92),
 (92, '2001-04-02 00:00:00.000000', 4, 2, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 57.019999999999996, 57.019999999999996, 57.019999999999996, 57.019999999999996, 55.94, 55.94, 55.94, 55.94, 57.019999999999996, 57.92, 60.08, 60.980000000000004, 62.96, 64.03999999999999, 64.94, 66.92, 64.03999999999999, 62.96, 62.06, 60.08, 57.92, 57.019999999999996, 55.94, 55.94),
 (93, '2001-04-03 00:00:00.000000', 4, 3, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 55.040000000000006, 57.019999999999996, 55.94, 55.94, 55.040000000000006, 53.96, 53.96, 53.96, 55.040000000000006, 55.94, 57.92, 59.0, 59

### Connect to a Previously Populated SQL Database and query

In [38]:
connection_string = 'sqlite:///cz_2010.sqlite'
engine = create_engine(connection_string)
Base = automap_base()
Base.prepare(engine, reflect=True)
cz_2010 = Base.classes.cz_2010_usc
session = Session(engine)

In [39]:
Base.classes.keys()

['cz_2010_usc']

In [40]:
WeatherData = Base.classes.cz_2010_usc

In [41]:
session = Session(engine)

In [42]:
first_row = session.query(WeatherData).first()

In [43]:
first_row.__dict__

{'READ_DT': datetime.datetime(2004, 1, 1, 0, 0),
 'READ_DT_DAY': 1,
 'READ_DT_MON': 1,
 'READ_DT_YEAR': 2004,
 'STATION': 'LOS-ANGELES-DOWNTOWN-USC',
 'STATION_NBR': 722874,
 'TEMP_F_0000': 51.8,
 'TEMP_F_0100': 48.2,
 'TEMP_F_0200': 46.04,
 'TEMP_F_0300': 46.04,
 'TEMP_F_0400': 44.96,
 'TEMP_F_0500': 42.980000000000004,
 'TEMP_F_0600': 42.980000000000004,
 'TEMP_F_0700': 42.08,
 'TEMP_F_0800': 42.08,
 'TEMP_F_0900': 44.06,
 'TEMP_F_1000': 48.92,
 'TEMP_F_1100': 53.06,
 'TEMP_F_1200': 55.040000000000006,
 'TEMP_F_1300': 57.92,
 'TEMP_F_1400': 60.08,
 'TEMP_F_1500': 62.06,
 'TEMP_F_1600': 62.06,
 'TEMP_F_1700': 60.980000000000004,
 'TEMP_F_1800': 58.82,
 'TEMP_F_1900': 56.84,
 'TEMP_F_2000': 55.040000000000006,
 'TEMP_F_2100': 53.6,
 'TEMP_F_2200': 53.24,
 'TEMP_F_2300': 51.8,
 'TYPE': 'CZ_2010',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x116ca8a58>,
 'id': 1}

In [44]:
inspector = inspect(engine)
for each in inspector.get_columns('cz_2010_usc'):
    print('Column Name: {} / Data Type: {}'.format(each['name'], each['type']))

Column Name: id / Data Type: INTEGER
Column Name: READ_DT / Data Type: DATETIME
Column Name: READ_DT_MON / Data Type: INTEGER
Column Name: READ_DT_DAY / Data Type: INTEGER
Column Name: READ_DT_YEAR / Data Type: INTEGER
Column Name: TYPE / Data Type: VARCHAR
Column Name: STATION / Data Type: VARCHAR
Column Name: STATION_NBR / Data Type: INTEGER
Column Name: TEMP_F_0000 / Data Type: FLOAT
Column Name: TEMP_F_0100 / Data Type: FLOAT
Column Name: TEMP_F_0200 / Data Type: FLOAT
Column Name: TEMP_F_0300 / Data Type: FLOAT
Column Name: TEMP_F_0400 / Data Type: FLOAT
Column Name: TEMP_F_0500 / Data Type: FLOAT
Column Name: TEMP_F_0600 / Data Type: FLOAT
Column Name: TEMP_F_0700 / Data Type: FLOAT
Column Name: TEMP_F_0800 / Data Type: FLOAT
Column Name: TEMP_F_0900 / Data Type: FLOAT
Column Name: TEMP_F_1000 / Data Type: FLOAT
Column Name: TEMP_F_1100 / Data Type: FLOAT
Column Name: TEMP_F_1200 / Data Type: FLOAT
Column Name: TEMP_F_1300 / Data Type: FLOAT
Column Name: TEMP_F_1400 / Data Type: 

In [45]:
inspector.get_table_names()

['cz_2010_usc']

In [46]:
inspector.get_primary_keys('cz_2010_usc')

/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.
  """Entry point for launching an IPython kernel.


['id']

In [47]:
conn = engine.connect()

In [48]:
query = """
        select * 
        from cz_2010_usc
        WHERE READ_DT_MON = 12
        """

In [49]:
df_december = pd.read_sql(query, conn)

In [50]:
df_december.head()

,id,READ_DT,READ_DT_MON,READ_DT_DAY,READ_DT_YEAR,TYPE,STATION,STATION_NBR,TEMP_F_0000,TEMP_F_0100,...,TEMP_F_1400,TEMP_F_1500,TEMP_F_1600,TEMP_F_1700,TEMP_F_1800,TEMP_F_1900,TEMP_F_2000,TEMP_F_2100,TEMP_F_2200,TEMP_F_2300
0,335,2004-12-01 00:00:00.000000,12,1,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,47.84,46.40,...,62.06,62.06,62.06,60.98,58.82,56.48,53.24,51.44,49.64,48.20
1,336,2004-12-02 00:00:00.000000,12,2,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,49.64,46.40,...,62.96,64.04,64.04,64.04,61.16,58.28,55.40,55.04,52.88,50.00
2,337,2004-12-03 00:00:00.000000,12,3,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,49.64,47.48,...,66.02,66.92,66.92,64.94,62.60,59.72,55.04,53.60,53.60,52.88
3,338,2004-12-04 00:00:00.000000,12,4,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,53.24,48.56,...,57.02,57.02,57.02,55.94,55.58,55.40,53.60,53.60,53.60,53.60
4,339,2004-12-05 00:00:00.000000,12,5,2004,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,46.40,51.80,...,50.00,51.08,50.00,50.00,48.92,48.56,50.00,49.64,47.84,46.40


### Query Using SQLAlchemy ORM

In [51]:
station_name = session.query(WeatherData.STATION).distinct().all()

In [52]:
station_number = session.query(WeatherData.STATION_NBR).distinct().all()

In [53]:
station_number

[(722874), (722880), (722970)]

In [54]:
station_name

[('LOS-ANGELES-DOWNTOWN-USC'), ('BURBANK-GLENDALE'), ('LONG-BEACH')]

In [55]:
#Query for station 722880 (BURBANK-GLENDALE) only.

TEMP = session.query(WeatherData.READ_DT_MON,
              WeatherData.READ_DT_DAY,
              WeatherData.READ_DT_YEAR,
              WeatherData.TEMP_F_0000, 
              WeatherData.TEMP_F_0100,
              WeatherData.TEMP_F_0200,
              WeatherData.TEMP_F_0300,
              WeatherData.TEMP_F_0400,
              WeatherData.TEMP_F_0500,
              WeatherData.TEMP_F_0600,
              WeatherData.TEMP_F_0700,
              WeatherData.TEMP_F_0800,
              WeatherData.TEMP_F_0900,
              WeatherData.TEMP_F_1000,
              WeatherData.TEMP_F_1100,
              WeatherData.TEMP_F_1200,
              WeatherData.TEMP_F_1300,
              WeatherData.TEMP_F_1400,
              WeatherData.TEMP_F_1500,
              WeatherData.TEMP_F_1600,
              WeatherData.TEMP_F_1700,
              WeatherData.TEMP_F_1800,
              WeatherData.TEMP_F_1900,
              WeatherData.TEMP_F_2000,
              WeatherData.TEMP_F_2100,
              WeatherData.TEMP_F_2200,
              WeatherData.TEMP_F_2300,
             ).filter((WeatherData.STATION_NBR == 722880) &
                     (WeatherData.READ_DT_MON >= 10) &
                     (WeatherData.READ_DT_MON <= 12)).all()

In [56]:
pd.read_json(pd.DataFrame(TEMP).to_json()).head()

,READ_DT_MON,READ_DT_DAY,READ_DT_YEAR,TEMP_F_0000,TEMP_F_0100,TEMP_F_0200,TEMP_F_0300,TEMP_F_0400,TEMP_F_0500,TEMP_F_0600,...,TEMP_F_1400,TEMP_F_1500,TEMP_F_1600,TEMP_F_1700,TEMP_F_1800,TEMP_F_1900,TEMP_F_2000,TEMP_F_2100,TEMP_F_2200,TEMP_F_2300
0,10,1,2005,69.44,67.64,68.36,67.64,66.74,66.02,64.40,...,84.38,85.82,84.02,82.40,80.42,78.62,76.64,73.22,71.60,71.42
1,10,2,2005,62.42,66.20,65.84,62.60,62.42,60.80,60.80,...,77.00,76.82,75.02,73.76,72.32,70.88,66.02,64.40,64.22,62.60
10,10,11,2005,60.62,64.22,62.60,62.42,60.44,57.20,57.02,...,75.38,76.82,75.56,73.76,71.42,69.44,66.02,64.22,62.42,60.80
11,10,12,2005,64.04,58.82,57.02,55.40,55.40,55.22,53.60,...,77.18,78.80,78.44,76.28,74.12,73.22,71.42,69.44,66.02,64.40
12,10,13,2005,73.04,60.80,60.44,57.20,57.02,55.40,55.58,...,93.20,93.20,93.02,92.12,90.32,88.88,83.48,78.44,75.02,73.40
